In [1]:
import numpy as np
import pandas as pd
import logging

import glob
import re
import pickle

from braindecode.datasets import TUH
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows, scale as multiply)

In [3]:
TUH_PATH = 'please insert actual path to data here'
N_JOBS = 2  # specify the number of jobs for loading and windowing
tuh = TUH(
    path=TUH_PATH,
    recording_ids=None,
    target_name=None,
    preload=False,
    add_physician_reports=False,
    n_jobs=N_JOBS,  
)


tuh. description.to_csv('./tuh_eeg_complete_description.csv')

reports = pd.read_csv('./tuh_eeg_complete_description.csv',  index_col=0)


In [4]:
impression = []
interpretation_idx = []
for irec in range(0,len(reports)): #69582
    text = str(reports.report[irec]).lower()
    if text.find('impression:') != -1:
        start = text.index('impression:') # 'IMPRESSION'
        if text[start:].find('clinical')!= -1:
            end = text[start:].index('clinical')
            endtext = start + end 
            impression.append(text[start:endtext])
        else:
            if text[start:].find('.\n\n')!= -1:
                end = text[start:].index('.\n\n')
                endtext = start + end 
                impression.append(text[start:endtext])
            else:
                end = text[start:].index('\n')
                endtext = start + end 
                impression.append(text[start:endtext])  
    elif text.find('impression/clinical') != -1:
        start = text.index('impression/clinical') # 'IMPRESSION'
       # if text[start:].find('CLINICAL')!= -1:
       #     end = text[start:].index('CLINICAL')
      ##      endtext = start + end 
        #impression.append(text[start:endtext])
       # else:
       
        if text[start:].find('.\n\n')!= -1:
            end = text[start:].index('.\n\n')
            endtext = start + end 
            impression.append(text[start:endtext])
        else:
            end = text[start:].index('\n')
            endtext = start + end 
            impression.append(text[start:endtext])     
    elif text.find('impression & clinical') != -1:
        start = text.index('impression & clinical') # 'IMPRESSION'
       # if text[start:].find('CLINICAL')!= -1:
       #     end = text[start:].index('CLINICAL')
      ##      endtext = start + end 
        #impression.append(text[start:endtext])
       # else:
       
        if text[start:].find('.\n\n')!= -1:
            end = text[start:].index('.\n\n')
            endtext = start + end 
            impression.append(text[start:endtext])
        else:
            end = text[start:].index('\n')
            endtext = start + end 
            impression.append(text[start:endtext])             
    elif text.find('impression') != -1:
        start = text.index('impression') # 'IMPRESSION'
        if text[start:].find('clinical')!= -1:
            end = text[start:].index('clinical')
            endtext = start + end 
            impression.append(text[start:endtext])
        else:
            if text[start:].find('.\n\n')!= -1:
                end = text[start:].index('.\n\n')
                endtext = start + end 
                impression.append(text[start:endtext])
            else:
                end = text[start:].index('\n')
                endtext = start + end 
                impression.append(text[start:endtext])           

        
    elif text.find('interpretation') != -1:
        start = text.index('interpretation') # 'IMPRESSION'
       # if text[start:].find('CLINICAL')!= -1:
       #     end = text[start:].index('CLINICAL')
      ##      endtext = start + end 
        #impression.append(text[start:endtext])
        interpretation_idx.extend([irec])
       # else:
        if text[start:].find('.\n\n')!= -1:
            end = text[start:].index('.\n\n')
            endtext = start + end 
            impression.append(text[start:endtext])
        else:
            end = text[start:].index('\n')
            endtext = start + end 
            impression.append(text[start:endtext])  
    else: impression.append('nan')      
        
print('len(impression)',len(impression)) #should be    69582
print('len(interpretation)',len(interpretation_idx)) #should be    12384

len(impression) 69582
len(interpretation) 12384


In [5]:
impression = [w.replace(':\n', ' ') for w in impression]

impression = [w.replace('.\n', '. ') for w in impression]
impression =[w.replace('\t', '') for w in impression]



reports['impression'] = impression

In [6]:
import re  # newlabelling
## find all recors with abnormal impression
abnormal = []

for irec in range(0,len(reports)): #69582
    text = str(reports.impression[irec])
    text = text.lower()

    regex1=re.compile(r"\babnormal")
    regex2=re.compile(r"\bno normal")
    regex3=re.compile(r"\bnot normal")
    regex4=re.compile(r"\bno abnormal")
    regex5=re.compile(r"\bnot abnormal")
    regex6=re.compile(r"\bnormal eeg")
    regex7=re.compile(r"\ba normal")
    regex8=re.compile(r"\babsence of a normal ")
    regex9=re.compile(r"\breplacement of a normal ")
    regex10=re.compile(r"\babsence of normal ")
    regex11=re.compile(r"\babnormal eeg")
    regex12=re.compile(r"\bno epileptiform abnormalities")  
    regex13=re.compile(r"\bis just outside of the range of normal")
    if (regex1.findall(text) != [] or regex2.findall(text) != [] or regex3.findall(text) != [] or regex8.findall(text) != [] or regex9.findall(text) != [] or regex10.findall(text) != [] or regex11.findall(text) != [] or regex13.findall(text) != [] ) and\
       (regex4.findall(text) == [] and regex5.findall(text) == []  and regex6.findall(text) == []  and \
         (regex7.findall(text) == [] or (regex7.findall(text) != [] and regex8.findall(text) != []) or (regex7.findall(text) != [] and regex11.findall(text) != []) or \
          (regex7.findall(text) != [] and regex9.findall(text) != []) or (regex7.findall(text) != [] and regex10.findall(text) != []) or\
         (regex7.findall(text) == [] and regex9.findall(text) == []))):
        abnormal.append(True)  #
         
    elif text.lower()== 'nan':
        abnormal.append('nan')    
    else: abnormal.append(False)    
    del regex1, regex2, regex3, regex4,regex5,regex6,regex7, regex8, regex9,regex10,regex11,regex12, regex13,text    
print(len(abnormal)) # should be 69582

reports['pathological'] = abnormal #69582

## find all recors with normal impression
normal = []
for irec in range(0,len(reports)):

    if reports.pathological[irec] == False:
        text = str(reports.impression[irec])
        text = text.lower()
        regex1=re.compile(r"\bnormal")
        regex2=re.compile(r"\bno abnormal")
        regex3=re.compile(r"\bnot abnormal")
        regex4=re.compile(r"\bno normal")
        regex5=re.compile(r"\bnot normal")
        regex6=re.compile(r"\babnormal eeg")
        if (regex1.findall(text) != [] or regex2.findall(text) != [] or regex3.findall(text) != []) and (regex4.findall(text) == [] and regex5.findall(text) == [] and regex6.findall(text) == []):
            normal.append(True)
        else:
            normal.append('no label')
        del regex1, regex2, regex3, regex4, regex5,regex6, text    
    if reports.pathological[irec] == 'nan':
        normal.append('nan')    
    if reports.pathological[irec] == True:   
        normal.append(False)
    #del text    
print(len(normal)) # should be 69582

reports['normal'] = list(normal)


69582
69582


In [42]:
occurrences = abnormal.count(True)
normal_occ = normal.count(True)
nokey = normal.count('no label')
nan = abnormal.count('nan')
print('Num records: ', len(abnormal)), print('abnormal :', occurrences) , print('normal :', normal_occ), print('no keyword found :', nokey), print('no Impression or Report :', nan)

# Num records:  69582
# abnormal : 38399
# normal : 13669
# no keyword found : 14565
# no Impression or Report : 2949
# 


Num records:  69582
abnormal : 38399
normal : 13669
no keyword found : 14565
no Impression or Report : 2949


(None, None, None, None, None)

In [43]:
reports.to_pickle("./reports_TUEG_labelled.pkl")

In [44]:
### selected chans and duration
label_all_tuh =  reports 
#pd.read_pickle("./reports_labelled.pkl")

In [45]:
artifact = pd.read_csv("./artifact_corpus_labels.csv") 
#artifact_set

eval_set= label_all_tuh
all_index = []
for iAB in range(len(artifact)):
    index = list(eval_set.loc[(eval_set['subject']== artifact['subject'][iAB])& (eval_set['year']== artifact['year'][iAB]) & (eval_set['month']== artifact['month'][iAB]) & (eval_set['day']== artifact['day'][iAB]) &  (eval_set['session']== artifact['session'][iAB]) ].index)
    all_index.extend(index)
    del index
len(all_index)   
eval_wo_arti= eval_set.drop(all_index)

In [46]:
tuab = pd.read_pickle("./TUHABnormaldescription.pkl")

In [ ]:
TUHAB = tuab# delete all subjects from TUHAB
eval2= eval_wo_arti

all_index = []
for iAB in range(len(TUHAB)):
    index = list(eval2.loc[(eval2['subject']== TUHAB['subject'][iAB])].index)
    all_index.extend(index)
    del index



eval_final= eval2.drop(all_index)



In [48]:
report_final= eval_final
new_labelsa = report_final[(report_final["pathological"] == True)]
new_labelsb = report_final[(report_final["pathological"] == False) & (report_final["normal"] == True) ]

new_labels = pd.concat([new_labelsa, new_labelsb])

len(new_labels) 

38674

In [50]:
df_sorted=new_labels.sort_values(by=['year', 'month', 'day', 'subject', 'session', 'segment'])


df_sorted['pathological']= df_sorted['pathological'].astype(int)


df_sorted= df_sorted.reset_index(drop=True)

In [51]:
df_sorted.to_pickle('./TUEG_reports_labelled_and_selected.pkl')

In [ ]:
## select data based on duration and channels

In [2]:
import time
import os
import mne
mne.set_log_level('ERROR')

from warnings import filterwarnings
filterwarnings('ignore')


from IPython.utils import io

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D


import torch
from torch.nn.functional import relu

from braindecode import EEGClassifier
from braindecode.training.losses import CroppedLoss
from braindecode.models import Deep4Net,ShallowFBCSPNet
from braindecode.util import set_random_seeds
from braindecode.models.util import to_dense_prediction_model, get_output_shape

from braindecode.datautil.windowers import create_fixed_length_windows
from braindecode.datautil.serialization import  load_concat_dataset

from braindecode.datasets import BaseConcatDataset


from functools import partial 
from skorch.callbacks import LRScheduler, EarlyStopping,Checkpoint, EpochScoring
from skorch.helper import predefined_split


    
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

import sys
sys.path.insert(0, '/home/kiessnek/TUHEEG_decoding/')
from ImbalancedDatasetSampler import ImbalancedDatasetSampler

from itertools import product

import GPUtil


#######################
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm


from braindecode.datasets.tuh import TUHAbnormal, TUH
from braindecode.datasets.base import BaseDataset,BaseConcatDataset    
from IPython.utils import io


import mne
import re
import os
import glob

import pandas as pd
import mne

def get_TUH_labeled(path_TUH, label_file, recording_ids=None):

        #from braindecode.datasets.tuh import TUH
       # add_physician_reports = True
        #self.label_file= label_file
        wanted_reports, paths = _load_labels(label_file) #,label_abnormal, label_normal

        all_datasets = []
        label_list =[]

        if recording_ids is None:
            recording_ids = len(paths)
           
        recording_reports = wanted_reports[:recording_ids]
        
        label_list = recording_reports['pathological'] #['PATHOLOGICAL']
        
       # recording_reports

        
        TUH_all_datasets = _load_TUH_labeled(reports_to_load=recording_reports, target_name=None, preload=False,
                         add_physician_reports=True)
        
             
        # adapt dataframe 
        # add dataset and pathologcial column so df. columns are identical between TUHEEG and TUHAbnormal

        all_datasets=BaseConcatDataset(TUH_all_datasets)
        target = label_list
        

             
        for d, y in zip(all_datasets.datasets, target):
            d.description['pathological'] = y
            d.target_name = 'pathological'
            d.target = d.description[d.target_name]
        all_datasets.set_description(pd.DataFrame([d.description for d in all_datasets.datasets]), overwrite=True)

       
        ds_name = ['TUHEEG'] * len(all_datasets.datasets)
        for ds, ys in zip(all_datasets.datasets, ds_name):
            ds.description['dataset'] = ys
            ds.target_name = 'dataset'
            ds.target = ds.description[d.target_name]
        all_datasets.set_description(pd.DataFrame([d.description for d in all_datasets.datasets]), overwrite=True)
        #all_datasets.set_description = (pd.DataFrame([d.description for d in all_datasets.datasets]),overwrite=True)

        all_datasets.description['dataset'] = 'TUHEEG'
        

        
        return all_datasets, label_list
        print(len(all_datasets), len(label_list))    

def _load_TUH_labeled(reports_to_load, target_name=None, preload=False,
                     add_physician_reports=True):
    base_datasets = []
    #descriptions  = reports_to_load
    for file_path_i, file_path in reports_to_load['path'].iteritems():
        # parse age and gender information from EDF header
        age,  gender = _parse_age_and_gender_from_edf_header(file_path) #test_report['age'][file_path_i]#
        #gender = test_report['gender'][file_path_i]
        raw = mne.io.read_raw_edf(file_path, preload=preload)
        # read info relevant for preprocessing from raw without loading it
        sfreq = raw.info['sfreq']
        n_samples = raw.n_times
        d = {
            'sfreq': float(sfreq),
            'n_samples': int(n_samples),
            'age': int(age),
            'gender': gender,
            }
        if add_physician_reports:
            physician_report = _read_physician_report(file_path)
            d['report'] = physician_report
        additional_description = pd.Series(d)
        #description = pd.concat(
        #   [reports_to_load, additional_description]) #descriptions.pop(file_path_i)
        description =reports_to_load.iloc[file_path_i]
        base_dataset = BaseDataset(raw, description,
                                   target_name=target_name)
        base_datasets.append(base_dataset)


    return base_datasets   
            
def _load_labels(filepath):


    reports = pd.read_pickle(filepath)
    
    wanted_reports = reports#
    paths = wanted_reports['path'].reset_index(drop=True)
    
    wanted_reports = wanted_reports.reset_index(drop=True)

    return wanted_reports, paths

def _parse_age_and_gender_from_edf_header(file_path):
    header = _read_edf_header(file_path)
    # bytes 8 to 88 contain ascii local patient identification
    # see https://www.teuniz.net/edfbrowser/edf%20format%20description.html
    patient_id = header[8:].decode("ascii")
    age = -1
    found_age = re.findall(r"Age:(\d+)", patient_id)
    if len(found_age) == 1:
        age = int(found_age[0])
    gender = "X"
    found_gender = re.findall(r"\s([F|M])\s", patient_id)
    if len(found_gender) == 1:
        gender = found_gender[0]
    return age, gender    

def _read_edf_header(file_path):
    f = open(file_path, "rb")
    header = f.read(88)
    f.close()
    return header

def _read_physician_report(file_path):
    directory = os.path.dirname(file_path)
    txt_file = glob.glob(os.path.join(directory, '**/*.txt'), recursive=True)
    # check that there is at most one txt file in the same directory
    assert len(txt_file) in [0, 1]
    report = ''
    if txt_file:
        txt_file = txt_file[0]
        # somewhere in the corpus, encoding apparently changed
        # first try to read as utf-8, if it does not work use latin-1
        try:
            with open(txt_file, 'r', encoding='utf-8') as f:
                report = f.read()
        except UnicodeDecodeError:
            with open(txt_file, 'r', encoding='latin-1') as f:
                report = f.read()
    return report

    

from tqdm.autonotebook import tqdm        


In [23]:
import time
#from combineTUHdatasets import *
print('load data ' )
start_load = time.time()
recording_ids=None
with io.capture_output() as captured:
    tuh, TUH_label_list = get_TUH_labeled(path_TUH='/data/datasets/TUH/EEG/tuh_eeg/', 
                                                                     label_file= './TUEG_reports_labelled_and_selected.pkl', 
                                                                     recording_ids=None) #CombineTUHDatasets.
end_load= time.time()
print(end_load -start_load)

load data 
1233.7344496250153


In [92]:
import os

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import mne

from braindecode.datasets import TUH
from braindecode.datautil.preprocess import preprocess, Preprocessor
from braindecode.datautil.windowers import create_fixed_length_windows
from braindecode.datautil.serialization import (
    save_concat_dataset, load_concat_dataset)

mne.set_log_level('ERROR')  # avoid messages everytime a window is extracted


def select_by_duration(ds, tmin=0, tmax=None):
    # determine length of the recordings and select based on tmin and tmax
    duration = ds.description.n_samples / ds.description.sfreq
    duration = duration[duration >= tmin]
    if tmax is None:
        tmax = np.inf
    duration = duration[duration <= tmax]
    split_ids = list(duration.index)
    splits = ds.split(split_ids)
    split = splits['0']
    return split


tmin = 15 * 60
tmax = None #n_max_minutes
tuh = select_by_duration(tuh, tmin, tmax)

In [93]:
short_ch_names = sorted([
    'A1', 'A2',
    'FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
    'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'FZ', 'CZ', 'PZ'])
ar_ch_names = sorted([
    'EEG A1-REF', 'EEG A2-REF',
    'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF',
    'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF',
    'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF',
    'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])
le_ch_names = sorted([
    'EEG A1-LE', 'EEG A2-LE',
    'EEG FP1-LE', 'EEG FP2-LE', 'EEG F3-LE', 'EEG F4-LE', 'EEG C3-LE',
    'EEG C4-LE', 'EEG P3-LE', 'EEG P4-LE', 'EEG O1-LE', 'EEG O2-LE',
    'EEG F7-LE', 'EEG F8-LE', 'EEG T3-LE', 'EEG T4-LE', 'EEG T5-LE',
    'EEG T6-LE', 'EEG FZ-LE', 'EEG CZ-LE', 'EEG PZ-LE'])
assert len(short_ch_names) == len(ar_ch_names) == len(le_ch_names)
ar_ch_mapping = {ch_name: short_ch_name for ch_name, short_ch_name in zip(
    ar_ch_names, short_ch_names)}
le_ch_mapping = {ch_name: short_ch_name for ch_name, short_ch_name in zip(
    le_ch_names, short_ch_names)}
ch_mapping = {'ar': ar_ch_mapping, 'le': le_ch_mapping}


def select_by_channels(ds, ch_mapping):
    split_ids = []
    for i, d in enumerate(ds.datasets):
        seta = set(ch_mapping[d.description.reference].keys())
        setb = set(d.raw.ch_names)
        if seta.issubset(setb):
            split_ids.append(i)
    return ds.split(split_ids)['0']


tuh = select_by_channels(tuh, ch_mapping)


In [95]:
tuh.description.to_pickle("./selected_labelled_reports_10min.pkl")

In [88]:
len(tuh.description[tuh.description['pathological']==1]),len(tuh.description[tuh.description['pathological']==0])

(20320, 6820)

In [76]:
tuh_15 = tuh.description

In [75]:
tuh.description.to_pickle("./selected_labelled_reports_15min.pkl")

In [3]:
tuh_selected= pd.read_pickle("./selected_labelled_reports_15min.pkl")

In [4]:
len(tuh_selected[tuh_selected['pathological']==1]),len(tuh_selected[tuh_selected['pathological']==0]), len(tuh_selected)

(10838, 4462, 15300)

In [4]:
df_sorted= tuh_selected.sort_values(by=['year', 'month', 'day', 'subject', 'session', 'segment'])


In [12]:
df_sorted2=df_sorted.reset_index(drop=True)

In [ ]:
#spilt in eval and train set SKF 10fold last fold

In [13]:
from sklearn.model_selection import StratifiedKFold 
skf = StratifiedKFold(n_splits=10,shuffle=False) 
target=df_sorted2.loc[:,'pathological']



In [14]:
df=df_sorted2
fold_no = 1
for train_index, test_index in skf.split(df, target):
    train = df.loc[train_index,:]
    test = df.loc[test_index,:]
    print('Fold',str(fold_no),'Class Ratio:',sum(test['pathological'])/len(test['pathological']))
    fold_no += 1


Fold 1 Class Ratio: 0.7084967320261438
Fold 2 Class Ratio: 0.7084967320261438
Fold 3 Class Ratio: 0.7084967320261438
Fold 4 Class Ratio: 0.7084967320261438
Fold 5 Class Ratio: 0.7084967320261438
Fold 6 Class Ratio: 0.7084967320261438
Fold 7 Class Ratio: 0.7084967320261438
Fold 8 Class Ratio: 0.7084967320261438
Fold 9 Class Ratio: 0.707843137254902
Fold 10 Class Ratio: 0.707843137254902


In [ ]:
train['train']= True
train
#13770 rows 

In [ ]:
test['train']= False
test

#2906

In [16]:
final = pd.concat([train, test]).reset_index(drop=True)

In [17]:
final_train= train.reset_index(drop=True)
final_eval=test.reset_index(drop=True)

In [18]:
final.to_pickle('./final_tuabex_description.pkl')
final_train.to_pickle('./final_train_tuabex_description.pkl')
final_eval.to_pickle('./final_eval_tuabex_description.pkl')

In [19]:
final.to_pickle('./final_tuabex_description.pkl')
final_train.to_pickle('./final_train_tuabex_description.pkl')
final_eval.to_pickle('./final_eval_tuabex_description.pkl')

In [ ]:
### preprocess and save data

In [5]:
import time

print('load data ' )
start_load = time.time()
recording_ids=None
with io.capture_output() as captured:
    dataset_train, TUH_label_list = get_TUH_labeled(path_TUH='/data/datasets/TUH/EEG/tuh_eeg/', 
                                                                     label_file= './final_train_tuabex_description.pkl', 
                                                                     recording_ids=None) 
end_load= time.time()
print(end_load -start_load)




load data 
1284.9824883937836


In [6]:
df_sorted= dataset_train.description.sort_values(by=['year', 'month', 'day', 'subject','session','segment'])
ids_sorted = list(df_sorted.index)
train_complete = dataset_train.split(ids_sorted)['0']

In [14]:
tuh = dataset_train
tuh_splits = tuh.split([[i] for i in range(len(dataset_train.datasets))])


duration = []


for ds, tuh_subset in tqdm(tuh_splits.items()):
    duration.append (tuh_subset.description.n_samples / tuh_subset.description.sfreq)

duration_train = train_complete.description
duration_train['duration']=list(duration)

duration_train.to_pickle('./duration_train_tuabextended.pkl')

In [7]:
import os

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import mne

from braindecode.datasets import TUH
from braindecode.datautil.preprocess import preprocess, Preprocessor
from braindecode.datautil.windowers import create_fixed_length_windows
from braindecode.preprocessing import (
    preprocess, Preprocessor, create_fixed_length_windows, scale as multiply)

mne.set_log_level('ERROR')  # avoid messages everytime a window is extracted

In [8]:
short_ch_names = sorted([
                'A1', 'A2', 'C3', 'C4', 'Cz', 'F3', 'F4', 'F7', 'F8',
                'Fp1', 'Fp2', 'Fz', 'O1', 'O2', 'P3', 'P4', 'Pz', 'T3',
                 'T4', 'T5', 'T6'
            ])
ar_ch_names = sorted([
    'EEG A1-REF', 'EEG A2-REF',
    'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF',
    'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF',
    'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF',
    'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])
le_ch_names = sorted([
    'EEG A1-LE', 'EEG A2-LE',
    'EEG FP1-LE', 'EEG FP2-LE', 'EEG F3-LE', 'EEG F4-LE', 'EEG C3-LE',
    'EEG C4-LE', 'EEG P3-LE', 'EEG P4-LE', 'EEG O1-LE', 'EEG O2-LE',
    'EEG F7-LE', 'EEG F8-LE', 'EEG T3-LE', 'EEG T4-LE', 'EEG T5-LE',
    'EEG T6-LE', 'EEG FZ-LE', 'EEG CZ-LE', 'EEG PZ-LE'])
assert len(short_ch_names) == len(ar_ch_names) == len(le_ch_names)
ar_ch_mapping = {ch_name: short_ch_name for ch_name, short_ch_name in zip(
    ar_ch_names, short_ch_names)}
le_ch_mapping = {ch_name: short_ch_name for ch_name, short_ch_name in zip(
    le_ch_names, short_ch_names)}
ch_mapping = {'ar': ar_ch_mapping, 'le': le_ch_mapping}



def custom_rename_channels(raw, mapping):
    # rename channels which are dependent on referencing:
    # le: EEG 01-LE, ar: EEG 01-REF
    # mne fails if the mapping contains channels as keys that are not present
    # in the raw
    reference = raw.ch_names[0].split('-')[-1].lower()
    assert reference in ['le', 'ref'], 'unexpected referencing'
    reference = 'le' if reference == 'le' else 'ar'
    raw.rename_channels(mapping[reference])


def custom_crop(raw, tmin=0.0, tmax=None, include_tmax=True):
    # crop recordings to tmin – tmax. can be incomplete if recording
    # has lower duration than tmax
    # by default mne fails if tmax is bigger than duration
    tmax = min((raw.n_times - 1) / raw.info['sfreq'], tmax)
    raw.crop(tmin=tmin, tmax=tmax, include_tmax=include_tmax)


n_max_minutes=21
tmin = 1 * 60
tmax = n_max_minutes * 60
sfreq = 100

preprocessors = [
    Preprocessor(custom_crop, tmin=tmin, tmax=tmax, include_tmax=False,
                 apply_on_array=False),

    Preprocessor(custom_rename_channels, mapping=ch_mapping,
                 apply_on_array=False),
    Preprocessor('pick_channels', ch_names=short_ch_names, ordered=True),
 
    Preprocessor(multiply, factor=1e6, apply_on_array=True),
    Preprocessor(np.clip, a_min=-800, a_max=800, apply_on_array=True),
    
    Preprocessor('set_eeg_reference', ref_channels='average', ch_type='eeg'),

    Preprocessor('resample', sfreq=sfreq),
    Preprocessor('set_meas_date', meas_date=None)
    
]

In [9]:
target = train_complete.description['pathological']



for d, y in zip(train_complete.datasets, target):
    d.description['pathological'] = y
    d.target_name = 'pathological'
    d.target = d.description[d.target_name]
train_complete.set_description(pd.DataFrame([d.description for d in train_complete.datasets]), overwrite=True)

In [ ]:
preprocess(
            concat_ds=train_complete,
            preprocessors=preprocessors,
            n_jobs=4, 
            save_dir='/home/data/preprocessed_TUABExtended/final_train/', 
        )

In [ ]:
train_complete.description.to_pickle('./TUABEX_trainset.pkl')

In [11]:
print('load data ' )
start_load = time.time()
recording_ids=None
with io.capture_output() as captured:
    dataset_eval, TUH_label_list = get_TUH_labeled(path_TUH='/data/datasets/TUH/EEG/tuh_eeg/', 
                                                                     label_file= './final_eval_tuabex_description.pkl', 
                                                                     recording_ids=None) #CombineTUHDatasets.
end_load= time.time()
print(end_load -start_load)

load data 
151.72280645370483


In [ ]:
eval_complete=dataset_eval

In [12]:
df_sorted= dataset_eval.description.sort_values(by=['year', 'month', 'day', 'subject','session','segment'])
ids_sorted = list(df_sorted.index)
eval_complete = dataset_eval.split(ids_sorted)['0']


target = eval_complete.description['pathological']



for d, y in zip(eval_complete.datasets, target):
    d.description['pathological'] = y
    d.target_name = 'pathological'
    d.target = d.description[d.target_name]
eval_complete.set_description(pd.DataFrame([d.description for d in eval_complete.datasets]), overwrite=True)

In [8]:
tuh = eval_complete
tuh_splits = tuh.split([[i] for i in range(len(eval_complete.datasets))])
#tuh_splits2 =  dict(list(tuh_splits.items())[23000:])

duration = []


for ds, tuh_subset in tqdm(tuh_splits.items()):
    duration.append (tuh_subset.description.n_samples / tuh_subset.description.sfreq)

duration_eval = eval_complete.description
duration_eval['duration']=list(duration)

duration_eval.to_pickle('./duration_eval_tuabextended.pkl')
#del dataset_eval

In [13]:
preprocess(
            concat_ds=eval_complete,
            preprocessors=preprocessors,
            n_jobs=4,  
            save_dir='/home/data/preprocessed_TUABExtended/final_eval/', 
        )

In [10]:
eval_complete.description.to_pickle('./TUABEX_evalset.pkl')

In [20]:
len(final_train), len(final_eval)

(13770, 1530)

In [ ]:
final_eval = eval_complete.description
final_train=train_complete.description

In [21]:
len(final_eval[final_eval['pathological']==1]), len(final_eval[final_eval['pathological']==0])

(1083, 447)

In [22]:
len(final_train[final_train['pathological']==1]), len(final_train[final_train['pathological']==0])

(9755, 4015)

In [2]:
#final.to_pickle('./final_tuabex_description.pkl')
final_train=pd.read_pickle('./final_train_tuabex_description.pkl')
final_eval=pd.read_pickle('./final_eval_tuabex_description.pkl')

In [2]:
final_train=pd.read_pickle('./TUABEX_trainset.pkl')
final_eval=pd.read_pickle('./TUABEX_evalset.pkl')

In [6]:
len(final_train[final_train['pathological']==1]),len(final_train[final_train['pathological']==0])

(9755, 4015)

In [7]:
len(final_eval[final_eval['pathological']==1]),len(final_eval[final_eval['pathological']==0])

(1083, 447)

In [9]:
len(final_train), len(final_eval)

(13770, 1530)

In [3]:
train_set= final_train
eval_set=final_eval

train_set['index']=train_set.index


eval_set['index']=eval_set.index

train_set_normal = train_set[train_set['pathological']==0]
train_set_patho = train_set[train_set['pathological']==1].reset_index(drop=True)

eval_set_normal = eval_set[eval_set['pathological']==0]
eval_set_patho = eval_set[eval_set['pathological']==1].reset_index(drop=True)

train_normal = train_set[(train_set['pathological'] == 0)]
train_patho = train_set[(train_set['pathological'] == 1)]
train_patho = train_patho.reset_index(drop=True)

In [4]:
import pickle
with open("./indices/indices_TUABEX_patho_train_set.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(train_set_patho['index']), fp) 
    
with open("./indices/indices_TUABEX_patho_eval_set.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(eval_set_patho['index']), fp) 

In [5]:
with open("./indices/indices_TUABEX_normal_train_set.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(train_set_normal['index']), fp) 
    
with open("./indices/indices_TUABEX_normal_eval_set.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(eval_set_normal['index']), fp) 

In [10]:
r = np.arange(len(train_patho))

r_eval =np.arange(len(eval_set_patho))

In [11]:
pidx_eval =r_eval[0:len(eval_set_patho):3]
pidx_train= r[0:len(train_patho):3]

In [12]:
len(pidx_eval), len(pidx_train)

(361, 3252)

In [13]:
list1= list(train_patho.index)
list2=list(pidx_train)
rest_train_patho= [item for item in list1 if item not in list2]



len(rest_train_patho)

r = np.arange(len(train_patho))
pidx_train2= r[0:len(rest_train_patho):9]

len(pidx_train2)

723

In [14]:
list1= list(eval_set_patho.index)
list2=list(pidx_eval)
rest_eval_patho= [item for item in list1 if item not in list2]



len(rest_eval_patho)

r = np.arange(len(eval_set_patho))
pidx_eval2= r[0:len(rest_eval_patho):9]

len(pidx_eval2)

81

In [66]:
#jfirst every third, then every 9th of the remaining recordings

In [16]:
train_patho_bal= train_patho.loc[sorted(list(pidx_train)+list(np.array(rest_train_patho)[pidx_train2]))]

eval_patho_bal= eval_set_patho.loc[sorted(list(pidx_eval)+list(np.array(rest_eval_patho)[pidx_eval2]))]

In [17]:
len(eval_patho_bal),len(eval_set_normal)  # jedes 9 

(442, 447)

In [18]:
len(train_patho_bal),len(train_set_normal)  # jedes 9 

(3975, 4015)

In [19]:
ind_bal_train = list(sorted(list(train_patho_bal['index'])+ list(train_set_normal['index'])))

trainbal = final_train.iloc[ind_bal_train]

len(trainbal[trainbal['pathological']==0]),len(trainbal[trainbal['pathological']==1])

(4015, 3975)

In [20]:
ind_bal_eval = list(sorted(list(eval_patho_bal['index'])+ list(eval_set_normal['index'])))

evalbal = final_eval.iloc[ind_bal_eval]

len(evalbal[evalbal['pathological']==0]),len(evalbal[evalbal['pathological']==1])

(447, 442)

In [34]:
import pickle
with open("./indices_TUABEx_balanced_train_patho.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(trainbal[trainbal['pathological']==1]['index']), fp)
    
with open("./indices_TUABEx_balanced_train_normal.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(trainbal[trainbal['pathological']==0]['index']), fp)    
    
with open("./indices_TUABEx_balanced_eval_patho.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(evalbal[evalbal['pathological']==1]['index']), fp)
    
with open("./indices_TUABEx_balanced_eval_normal.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(evalbal[evalbal['pathological']==0]['index']), fp)    

In [35]:
with open("./indices_TUABEx_balanced_train_set.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(trainbal['index']), fp)   

In [36]:
with open("./indices_TUABEx_balanced_eval_set.pkl", "wb") as fp:   #Pickling
    pickle.dump(list(evalbal['index']), fp)  

In [27]:
trainbal.to_pickle('./TUABEX_train_set_balanced.pkl')

In [28]:
evalbal.to_pickle('./TUABEX_eval_set_balanced.pkl')